# L4: Continual Pre-Training Demo

In [2]:
!pip install datasets
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

In [3]:
from pprint import pprint
import math
import wandb

import datasets
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [4]:
ds = load_dataset('ai4bharat/sangraha', data_files="https://huggingface.co/datasets/ai4bharat/sangraha/resolve/main/verified/tam/data-0.parquet")['train']
print(ds)

README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

data-0.parquet:   0%|          | 0.00/358M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doc_id', 'text', 'type'],
    num_rows: 149796
})


In [9]:
%%bash
huggingface-cli login --token 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `dlp_practice` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `dlp_practice`


In [12]:
model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
print(f'Vocab Size: {tokenizer.vocab_size}')
print(f'Context Length: {tokenizer.model_max_length}')

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Vocab Size: 128000
Context Length: 131072


In [13]:
tokenizer.model_max_length = 1024
tokenizer.pad_token = tokenizer.eos_token

In [14]:
example = ds[1]
num_words = len(example['text'].split())
print(f'Number of words: {num_words}')

Number of words: 47


In [15]:
input_ids = tokenizer(example['text'])['input_ids']
print(f'Number of tokens: {len(input_ids)}')

Number of tokens: 521


In [16]:
print(example['text'])

செய்முறைஃ
பச்சரிசி மற்றும் பச்சைப்பயறை ஒன்றாக சேர்த்து ஒரு மணி நேரம் ஊற வைக்கவும். ஊறிய அரிசி, பயறுடன், தேங்காய் துருவல், காய்ந்த மிளகாய், பெருங்காயத்தூள், இஞ்சி, கொத்தமல்லி, கறிவேப்பிலை, உப்பு சேர்த்து தோசை மாவு பதத்தில் அரைத்து கொள்ளவும்.
 அடுப்பில் தோசைக்கல்லை வைத்து எண்ணெய் ஊற்றி காய்ந்ததும் அரைத்து வைத்திருக்கும் மாவை ஊற்றி சுட்டு எடுக்கவும். சுவையான பச்சை பயறு தோசை ரெசிபி ரெடி. 


In [17]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(tokens[0:47])
tokenizer.decode(input_ids)

['<|begin_of_text|>', 'à®', 'ļ', 'à¯', 'Ĩ', 'à®', '¯', 'à¯įà®', '®', 'à¯ģ', 'à®', '±', 'à¯', 'Ī', 'à®', 'ĥ', 'Ċ', 'à®', 'ª', 'à®', 'ļ', 'à¯įà®', 'ļ', 'à®', '°', 'à®¿à®', 'ļ', 'à®¿', 'Ġà®', '®', 'à®', '±', 'à¯įà®', '±', 'à¯ģ', 'à®', '®', 'à¯į', 'Ġà®', 'ª', 'à®', 'ļ', 'à¯įà®', 'ļ', 'à¯', 'Ī', 'à®']


'<|begin_of_text|>செய்முறைஃ\nபச்சரிசி மற்றும் பச்சைப்பயறை ஒன்றாக சேர்த்து ஒரு மணி நேரம் ஊற வைக்கவும். ஊறிய அரிசி, பயறுடன், தேங்காய் துருவல், காய்ந்த மிளகாய், பெருங்காயத்தூள், இஞ்சி, கொத்தமல்லி, கறிவேப்பிலை, உப்பு சேர்த்து தோசை மாவு பதத்தில் அரைத்து கொள்ளவும்.\n அடுப்பில் தோசைக்கல்லை வைத்து எண்ணெய் ஊற்றி காய்ந்ததும் அரைத்து வைத்திருக்கும் மாவை ஊற்றி சுட்டு எடுக்கவும். சுவையான பச்சை பயறு தோசை ரெசிபி ரெடி. '

In [22]:
def tokenize(example):
  example = tokenizer(example['text'], padding=False, truncation=True)
  return example

In [23]:
tokenized_ds = ds.map(tokenize, batched=True, num_proc = 12, remove_columns=['doc_id', 'text', 'type'])
print(tokenized_ds)

Map (num_proc=12):   0%|          | 0/149796 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 149796
})


In [24]:
def concatenate_and_chunk(examples):
  pass

In [25]:
ds_chunked = ds.map(concatenate_and_chunk, batch_size=1000, batched=True, num_proc=12, remove_columns=['doc_id', 'text', 'type'])
ds_chunked.save_to_disk('tamil_ds')

Map (num_proc=12):   0%|          | 0/149796 [00:00<?, ? examples/s]

Saving the dataset (0/3 shards):   0%|          | 0/149796 [00:00<?, ? examples/s]

In [26]:
ds_chunked = load_from_disk('tamil_ds')
print(ds_chunked)

Dataset({
    features: ['doc_id', 'text', 'type'],
    num_rows: 149796
})


In [27]:
ds_split = ds_chunked.train_test_split(test_size=0.001, seed=42)
print(ds_split)

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'text', 'type'],
        num_rows: 149646
    })
    test: Dataset({
        features: ['doc_id', 'text', 'type'],
        num_rows: 150
    })
})


In [28]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [29]:
model = AutoModelForCausalLM.from_pretrained(model_id, pad_token_id=tokenizer.eos_token_id)

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [30]:
configuration = model.config
print(configuration)

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "meta-llama/Llama-3.2-1B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 16,
  "num_key_value_heads": 8,
  "pad_token_id": 128001,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "vocab_size": 128256
}



In [31]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
     

In [32]:
num_parameters = 0
for param in model.parameters():
  num_parameters += param.numel()
print(f'Number of parameters: {num_parameters}')

Number of parameters: 1235814400


In [33]:
print(model.dtype)

torch.float32


In [34]:
mem_in_gb = num_parameters * 4 / 1024 / 1024 / 1024
print(f'Memory in GB: {mem_in_gb}')

Memory in GB: 4.603767395019531


In [35]:
print(model.get_memory_footprint()/1e9)

4.943259776


In [36]:
param_model = num_parameters*4/ 1e9
adam_opt = 3*param_model
kernel = 1
bs = 1
print(f'Total Memory Requirement per sample: {(param_model + adam_opt + kernel)*bs} GB')

Total Memory Requirement per sample: 20.7730304 GB


# L5: Continual Pre-training Demo PEFT LoRA

In [37]:
prompt = "I was reading Feyman's lecture on physics. He talks about "
inputs = tokenizer(prompt, return_tensors='pt', padding=True)
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, top_p=0.95, top_k=10)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["I was reading Feyman's lecture on physics. He talks about 2 things. One is the importance of a theory, and the other is the importance of a measurement. He says that if you can't measure something, you can't really know it.\nI think that's true. But I think that if you"]

In [39]:
from peft import LoraConfig, TaskType, LoraModel
lora_config = LoraConfig(r=16, target_modules=["q_proj", "v_proj"], task_type=TaskType.CAUSAL_LM, inference_mode=False, lora_alpha=32, lora_dropout=0.05)

In [42]:
from peft import get_peft_model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


/usr/local/lib/python3.11/dist-packages/peft/mapping.py:185: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'meta-llama/Llama-3.2-1B' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [43]:
print(lora_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
            (layers): ModuleList(
              (0-15): 16 x LlamaDecoderLayer(
                (self_attn): LlamaSdpaAttention(
                  (q_proj): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDi

In [44]:
num_parameters_lora = 0
for param in lora_model.parameters():
  if param.requires_grad:
    num_parameters_lora += param.numel()
print(f'Number of parameters of LORA model: {num_parameters_lora}')

Number of parameters of LORA model: 1703936


In [45]:
lora_model.peft_config

{'default': LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=16, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}

In [48]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="lora_llama_1b_ct", eval_strategy="steps", eval_steps=100, num_train_epochs=1, per_device_train_batch_size=2, per_device_eval_batch_size=2, bf16=False, fp16=True, tf32=False, gradient_accumulation_steps=1, adam_beta1=0.9, adam_beta2=0.999, learning_rate=2e-5, weight_decay=0.01, logging_dir='logs', logging_strategy='steps', logging_steps=100, save_steps=100, save_total_limit=20, report_to='none')

In [50]:
trainer = Trainer(model=lora_model, args=training_args, train_dataset=ds_split['train'], eval_dataset=ds_split['test'], data_collator=data_collator)

# L6: PEFT + Optimization

In [51]:
from transformers import BitsAndBytesConfig
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

In [53]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 711.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [2]:
from transformers import AutoModelForCausalLM

In [3]:
model_8bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, pad_token_id=tokenizer.eos_token_id, device_map='auto')

NameError: name 'model_id' is not defined

In [ ]:
print(model_8bit)